In [3]:
%load_ext autoreload
%autoreload 1

#  https://colab.research.google.com/notebooks/data_table.ipynb
from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive
drive.mount('/content/gdrive')

# https://stackoverflow.com/questions/52733786/how-to-import-custom-modules-in-google-colab
import sys
sys.path.append('/content/gdrive/MyDrive/SEMRUSH/')

%aimport my

import sys
import numpy as np
import pandas as pd
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import pickle

import my
from my import p

pd.set_option("max_colwidth", 35)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from IPython.display import HTML, display

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
     
 # номер или название эксперимента
# n_exp = '1/'
dir_out = "/content/gdrive/MyDrive/SEMRUSH/out/"
dir_data = '/content/gdrive/MyDrive/SEMRUSH/data/'
os.makedirs(dir_out, exist_ok = True)
RANDOM_STATE = 34

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Mounted at /content/gdrive


In [4]:
stat = pd.read_parquet(dir_data+'stat.p')
p(stat.shape)
stat[:5]

(13040643, 23) 
 ~


,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y,root_domain,...,path_url,path_url_ref,cat_domain,cat_ref,pair_domain,pair_root_domain,pair_ltd_domain,pair_cat_domain,pair_url,pair_path_url
0,0,224312,1177617984,189763,1,531346,1,0,0,157605,...,1317010,3900817,65,0,1897630000001,1576050000001,2330000263,650000000,5313460000001,13170103900817
1,0,224312,1177618316,804359,1,2108910,1,1,0,670068,...,1531109,3900817,65,0,8043590000001,6700680000001,2310000263,650000000,21089100000001,15311093900817
2,0,224312,1177618348,988665,1,2549286,1,2,0,823669,...,1317010,3900817,65,0,9886650000001,8236690000001,2310000263,650000000,25492860000001,13170103900817
3,0,224312,1177618382,804359,1,2108910,1,3,0,670068,...,1531109,3900817,65,0,8043590000001,6700680000001,2310000263,650000000,21089100000001,15311093900817
4,0,224312,1177618416,141092,988665,3538739,2549382,4,0,117227,...,1812412,1317010,65,65,1410920988665,1172270823669,850000231,650000065,35387392549382,18124121317010


In [5]:
N_SPLIT_STAT = 3
N_SPLIT_TRAIN = 3

device_folds = my.stratify_device_group_split(
    stat,
    n_splits_level1=N_SPLIT_STAT,
    n_splits_level2=N_SPLIT_TRAIN,
    random_state=RANDOM_STATE)

device_folds

[{'level_1': (array([224312, 278965, 105686, ..., 121914, 178818, 247351], dtype=int32),
   array([131420, 131803, 219147, ..., 169873, 243491, 232250], dtype=int32)),
  'level_2': [(array([131803, 131698, 210918, ..., 128263, 103076, 103205], dtype=int32),
    array([131420, 219147, 255315, ..., 169873, 243491, 232250], dtype=int32)),
   (array([131420, 219147, 131698, ..., 169873, 243491, 232250], dtype=int32),
    array([131803, 169595, 106892, ..., 165109, 217932, 128263], dtype=int32)),
   (array([131420, 131803, 219147, ..., 169873, 243491, 232250], dtype=int32),
    array([131698, 210918, 235354, ..., 212253, 103076, 103205], dtype=int32))]},
 {'level_1': (array([131420, 105686, 145058, ..., 243491, 232250, 247351], dtype=int32),
   array([224312, 278965, 137810, ..., 186153, 161419, 121914], dtype=int32)),
  'level_2': [(array([224312, 278965, 226014, ..., 138455, 186153, 161419], dtype=int32),
    array([137810, 208598, 116675, ..., 149393, 140686, 121914], dtype=int32)),
   (

In [6]:
my.save_pickle(dir_data+\
    f'folds_{N_SPLIT_STAT}to{N_SPLIT_TRAIN}.pickle', device_folds)

In [7]:
device_folds[0]['level_1'][0].shape,\
device_folds[0]['level_1'][1].shape,\
device_folds[0]['level_2'][0][0].shape,\
device_folds[0]['level_2'][0][1].shape

((133296,), (66704,), (44461,), (22243,))

In [8]:
for i, fold in enumerate(device_folds):
  print('fold=',i)
  stat_ids = fold['level_1'][0]
  part_stat = stat[stat.device_id.isin(stat_ids)]\
      .sort_values(['event_group_id','pos']).reset_index(drop=True)
  p(part_stat.shape)
  sts = my.calc_stats(part_stat)

  name = f'sts_split_{N_SPLIT_STAT}_fold_{i}.pickle'
  print('file= ', name)
  my.save_pickle(dir_data+name, sts)
  del sts
  gc.collect()

  

fold= 0
(8701781, 23) 
 ~
file=  sts_split_3_fold_0.pickle
fold= 1
(8721306, 23) 
 ~
file=  sts_split_3_fold_1.pickle
fold= 2
(8658199, 23) 
 ~
file=  sts_split_3_fold_2.pickle


In [7]:
fold_1=0
fold_2=0
stat_ids, train_ids, val_ids = my.get_devices_from_folds(device_folds, fold_1, fold_2)
stat_ids.shape, train_ids.shape, val_ids.shape

((100017,), (74981,), (25002,))

In [ ]:
stat[:3]

,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y,root_domain,...,path_url,path_url_ref,cat_domain,cat_ref,pair_domain,pair_root_domain,pair_ltd_domain,pair_cat_domain,pair_url,pair_path_url
0,0,224312,1177617984,189763,1,531346,1,0,0,157605,...,1317010,3900817,65,0,1897630000001,1576050000001,2330000263,650000000,5313460000001,13170103900817
1,0,224312,1177618316,804359,1,2108910,1,1,0,670068,...,1531109,3900817,65,0,8043590000001,6700680000001,2310000263,650000000,21089100000001,15311093900817
2,0,224312,1177618348,988665,1,2549286,1,2,0,823669,...,1317010,3900817,65,0,9886650000001,8236690000001,2310000263,650000000,25492860000001,13170103900817


In [9]:
stat = stat[stat.device_id.isin(stat_ids)]\
  .sort_values(['event_group_id','pos']).reset_index(drop=True)
p(stat.shape ,stat.columns)
stat[:3]

(6501209, 23) 
 ~


,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y,root_domain,...,path_url,path_url_ref,cat_domain,cat_ref,pair_domain,pair_root_domain,pair_ltd_domain,pair_cat_domain,pair_url,pair_path_url
0,0,224312,1177617984,189763,1,531346,1,0,0,157605,...,1317010,3900817,65,0,1897630000001,1576050000001,2330000263,650000000,5313460000001,13170103900817
1,0,224312,1177618316,804359,1,2108910,1,1,0,670068,...,1531109,3900817,65,0,8043590000001,6700680000001,2310000263,650000000,21089100000001,15311093900817
2,0,224312,1177618348,988665,1,2549286,1,2,0,823669,...,1317010,3900817,65,0,9886650000001,8236690000001,2310000263,650000000,25492860000001,13170103900817


In [12]:
%%time
sts = my.calc_stats(stat)
sts.keys()

CPU times: user 1min 7s, sys: 346 ms, total: 1min 8s
Wall time: 1min 7s


In [ ]:
my.save_pickle(dir_data+\
            f'sts_split_{N_SPLIT_STAT}_fold_{fold_1}.pickle',sts)